# Last API with pylast

[Oficial Website](https://www.last.fm/api/)

[PyLast Repo](https://github.com/pylast)

In [1]:
import requests

In [2]:
import pandas as pd 
import numpy as np 
import pylast

import os
import json
import sys

sys.path.append('../scripts/') #Yes, i'm on windows

from create_last_database import User

In [3]:
API_KEY = input()
API_SECRET = input()

network = pylast.LastFMNetwork(api_key=API_KEY, api_secret=API_SECRET)

### Writting users by network

In [4]:
known_users = ['arcticmonkeys-','angeloky','allanxguimaraes', 'angeloky']
write_users = User(network)
write_users.write_users_names(known_users, '../data/users.csv', 5, 8)

Level 1 compĺeted


Now we have a small part of tha data (100 random users from the preview 44 thousand users). We've collected only some of the possible information

In [118]:
if not os.path.exists('users_info.json'):
    with open('users_info.json', 'w') as f:
        json.dump([], f)

with open('users_info.json', 'r+') as f: 
    data = json.load(f)
    data.extend(users_info)
    f.seek(0)
    json.dump(data, f)